## Hipótese 1
A primeira hipótese é que usuários que possuem um `skip_rate` (taxa de músicas puladas) muito alto estão insatisfeitos com as recomendações do algoritmo ou com o catálogo. Se a pessoa passa mais tempo pulando música do que ouvindo, a chance de ela cancelar a assinatura deve ser maior.

## Hipótese 2
A segunda hipótese foca nos usuários do plano Free. O número de `ads_listened_per_week` (anúncios ouvidos) tem um limite de tolerância. Usuários bombardeados por muitos anúncios tendem a abandonar a plataforma (Churn), enquanto aqueles que não ouvem anúncios (Premium) tendem a ficar.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

df = pd.read_csv('data.csv')

df_processed = pd.get_dummies(df, columns=['gender', 'country', 'subscription_type', 'device_type'], drop_first=True)

X = df_processed.drop('is_churned', axis=1)
y = df_processed['is_churned']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = DecisionTreeClassifier(random_state=42, max_depth=5)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

print(f"Acurácia:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precisão:  {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")

importancia = pd.Series(modelo.feature_importances_, index=X.columns).sort_values(ascending=False)

print("\nFatores que mais causam churn (em ordem):")
print(importancia.head(5))

Acurácia:  0.7456
Precisão:  0.1818
Recall:    0.0050
F1-Score:  0.0097

Fatores que mais causam churn (em ordem):
user_id                 0.241973
listening_time          0.213660
skip_rate               0.206623
songs_played_per_day    0.128761
gender_Other            0.086222
dtype: float64


# Validação das Teorias

## Hipótese 1: 

A hipótese se provou parcialmente validada.

O `skip_rate` ocupou a terceira posição no ranking de importância (20,7%), confirmando que é um fator significativo para prever churn. Esse resultado sustenta nossa teoria inicial: usuários que pulam muitas músicas apresentam padrões comportamentais distintos de potenciais canceladores.

O algoritmo identificou que a taxa de músicas puladas funciona como um indicador válido de insatisfação, embora não seja o fator dominante. Outros padrões comportamentais (tempo de audição total e quantidade de músicas tocadas) apresentam relevância comparable.

## Hipótese 2:

A hipótese não foi validada.

Contrariamente ao esperado, `ads_listened_per_week` não apareceu entre os top-5 features mais importantes. Similarmente, `subscription_type` não foi incluído nas variáveis que o modelo considerou relevantes para a predição.

2 cenários podem explicar este resultado:

1. Usuários em diferentes planos podem estar distribuídos de forma que os anúncios não discriminam efetivamente entre churners e retentores

2. Variáveis como `listening_time` e `skip_rate` podem estar capturando o efeito dos anúncios de forma indireta, relegando a importância direta desta métrica


## Performance geral:

Um aspecto crucial que limita a confiabilidade das validações é o recall extremamente baixo (0,5%). O modelo identifica apenas 1 em cada 200 usuários que realmente efetuam churn. Isso significa que:

- As predições positivas (churn) do modelo são pouco confiáveis
- A importância das features pode estar enviesada para capturar padrões na classe majoritária (não-churn)
- Conclusões sobre hipóteses devem ser tratadas como indicativas, não definitivas
